In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from elastic_rods import EnergyType
from bending_validation import suppress_stdout as so

In [ ]:
import numpy as np

In [ ]:
width = 0.1
thickness = width
cross_section = [thickness, width]

In [ ]:
input_path = '../../data/sphere_cap_0.3.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
width = 2*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]

### Initialization

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
curr_um.targetDeploymentHeight = thickness * 1

In [ ]:
curr_um.energy(UmbrellaEnergyType.Elastic)

In [ ]:
curr_um.set_design_parameter_config(True, False)

### Parameters

In [ ]:
view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
view.averagedMaterialFrames = True
view.show()

### Gradient Validation

In [ ]:
import umbrella_finite_diff

In [ ]:
import importlib
importlib.reload(umbrella_finite_diff)

In [ ]:
init_dof = curr_um.getExtendedDoFs()

In [ ]:
perturbation = np.random.uniform(-0.1, 0.1, curr_um.numExtendedDoF())

In [ ]:
curr_um.setExtendedDoFs(init_dof + perturbation)

In [ ]:
view.update()

In [ ]:
direction = np.random.uniform(-1, 1, curr_um.numExtendedDoF())

In [ ]:
curr_um.updateSourceFrame()

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 100

In [ ]:
curr_um.targetDeploymentHeight = thickness

In [ ]:
curr_um.energy(umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment)

In [ ]:
import numpy.linalg as la

In [ ]:
la.norm(curr_um.gradient(umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment, variableDesignParameters=True))

In [ ]:
curr_um.energy(umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment)

In [ ]:
curr_um.energy(umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Elastic)

In [ ]:
umbrella_finite_diff.gradient_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment, etype=elastic_rods.EnergyType.Full, direction = direction, variableDesignParameters=True)

In [ ]:
umbrella_finite_diff.gradient_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Full, etype=elastic_rods.EnergyType.Full, direction = direction, variableDesignParameters=True)

In [ ]:
umbrella_finite_diff.gradient_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Elastic, etype=elastic_rods.EnergyType.Full, direction = direction, variableDesignParameters=True)